# Capítulo 05 — Pré-Treinamento: O Nascimento da Inteligência

Neste capítulo, vamos tirar o nosso GPTMini da inércia. Vamos alimentá-lo com dados e observar o momento exato em que ele deixa de escolher palavras aleatórias e começa a entender os padrões da nossa língua.

--- 
### 🏭 A Fábrica de Predição
O pré-treinamento é a etapa mais pesada. É aqui que o modelo aprende a gramática, o vocabulário e a lógica do mundo através da predição do próximo token.

![Pipeline de Treinamento](./infograficos/01-pipeline-treinamento.png)

## 1. Setup e Configuração

In [ ]:
# ============================================================
# Setup do repositório no Colab
# ============================================================
import os
import sys

REPO_URL = "https://github.com/vongrossi/fazendo-um-llm-do-zero.git"
REPO_DIR = "fazendo-um-llm-do-zero"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL}

os.chdir(REPO_DIR)
sys.path.append(os.getcwd())
print("Diretório atual:", os.getcwd())


In [ ]:
!pip -q install -r 05-pre-treinamento/requirements.txt

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import math
import random
import numpy as np
from lib.gptmini import GPTConfig, GPTMini

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Oficina pronta! Usando: {device}")

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)


## 2. Preparação do Dataset

Vamos usar um pequeno conjunto de frases sobre gatos e cachorros para que possamos monitorar o aprendizado de forma rápida e clara.

In [ ]:
text = """
o gato subiu no telhado
o cachorro subiu no sofa
o gato dormiu no sofa
o cachorro dormiu no tapete
o gato pulou no muro
o cachorro pulou o portao
""".strip().lower()

tokens = text.split()
vocab = sorted(set(tokens))
stoi = {t:i for i,t in enumerate(vocab)}
itos = {i:t for t,i in stoi.items()}
vocab_size = len(vocab)
encoded = [stoi[t] for t in tokens]

def build_dataset(token_ids, context_size=5):
    X, Y = [], []
    for i in range(len(token_ids) - context_size):
        X.append(token_ids[i : i + context_size])
        Y.append(token_ids[i + 1 : i + context_size + 1])
    return torch.tensor(X).to(device), torch.tensor(Y).to(device)

context_size = 5
X, Y = build_dataset(encoded)
print(f"Vocabulário: {vocab_size} palavras | Amostras: {len(X)}")

## 3. O Estado do Caos (Antes do Treino)

Vamos ver o que o modelo gera agora, com seus pesos puramente aleatórios.

In [ ]:
config = GPTConfig(vocab_size=vocab_size, context_size=5, d_model=64, n_heads=4, n_layers=2)
model = GPTMini(config).to(device)

def generate(model, start_text, max_new=5):
    model.eval()
    tokens = [stoi[t] for t in start_text.split() if t in stoi]
    idx = torch.tensor(tokens).unsqueeze(0).to(device)
    for _ in range(max_new):
        logits, _ = model(idx[:, -5:])
        next_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        idx = torch.cat([idx, next_id], dim=1)
    return " ".join([itos[i.item()] for i in idx[0]])

print("🎲 Geração Aleatória:")
print(generate(model, "o gato"))

## 4. O Loop de Treinamento

Agora vamos ensinar o modelo através da **Cross Entropy**. Ele vai comparar sua predição com o alvo real e ajustar seus pesos via **Backpropagation**.

![Loop Treino](./infograficos/03-loop-treinamento.png)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
loss_history = []

print("🔨 Treinando o cérebro do modelo...")
model.train()
for step in range(501):
    idx = torch.randint(0, X.size(0), (8,))
    xb, yb = X[idx], Y[idx]
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_history.append(loss.item())
    if step % 100 == 0: print(f"Step {step:03d} | Loss: {loss.item():.4f}")

plt.figure(figsize=(8, 3))
plt.plot(loss_history, color='#1A73E8')
plt.title("Curva de Aprendizado (Cross Entropy Loss)")
plt.show()

## 5. Ordem a partir do Caos (Depois do Treino)

Agora o modelo já deve ser capaz de completar frases sobre os animais do nosso dataset de forma coerente.

In [ ]:
print("✨ Geração Treinada:")
print(generate(model, "o gato"))
print(generate(model, "o cachorro"))

## 6. Estratégias de Geração Avançadas

O modelo não precisa ser sempre "robótico". Podemos usar a **Temperatura** para torná-lo mais criativo ou o **Top-P** para focar apenas nas melhores opções.

![Decoding Strategies](./infograficos/04-decoding-strategies.png)

In [ ]:
@torch.no_grad()
def generate_creative(model, start_text, temp=1.0):
    model.eval()
    tokens = [stoi[t] for t in start_text.split() if t in stoi]
    idx = torch.tensor(tokens).unsqueeze(0).to(device)
    for _ in range(5):
        logits, _ = model(idx[:, -5:])
        logits = logits[:, -1, :] / temp
        probs = F.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        idx = torch.cat([idx, next_id], dim=1)
    return " ".join([itos[i.item()] for i in idx[0]])

print("🌡️ Baixa Temperatura (Focado):")
print(generate_creative(model, "o gato", temp=0.1))

print("\n🔥 Alta Temperatura (Aleatório/Criativo):")
print(generate_creative(model, "o gato", temp=1.5))

## 7. Persistência: Salvando o Checkpoint

Para não perdermos o progresso, salvamos o estado das matrizes de pesos.

![Checkpoints](./infograficos/05-checkpoints.png)

In [ ]:
torch.save(model.state_dict(), "gpt_checkpoint.pt")
print("✅ Checkpoint salvo como 'gpt_checkpoint.pt'!")

## 🏁 Conclusão

Você acabou de ensinar um GPT a aprender linguagem. Viu como ele sai do caos aleatório para a ordem gramatical e como podemos controlar sua criatividade.

No próximo capítulo, vamos levar este modelo para a **especialização**, aprendendo como fazer o **Fine-Tuning** para tarefas específicas como classificação de sentimentos e detecção de spam.